In [2]:
import tensorflow as tf
import keras
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from keras.preprocessing.image import ImageDataGenerator

In [3]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=10,
    fill_mode='nearest',
    featurewise_center=True,
    featurewise_std_normalization=True,
    vertical_flip=True,
    shear_range=0.2,
    zoom_range=0.2,
    brightness_range=(0.4,0.6),
    horizontal_flip=True,
    validation_split=0.2)
test_datagen=ImageDataGenerator(
    rescale=1./255
)

In [4]:
train_generator=train_datagen.flow_from_directory(
    'C:\\Users\\Sivaprasath\\Desktop\\RAMANAN\\emotion detextion\\archive\\train',
    target_size=(48,48),
    batch_size=64,
    color_mode='grayscale',
    class_mode='binary'
)

Found 28709 images belonging to 7 classes.


In [5]:
test_generator=test_datagen.flow_from_directory(
    "C:\\Users\\Sivaprasath\\Desktop\\RAMANAN\\emotion detextion\\archive\\test",
    target_size=(48,48),
    batch_size=64,
    color_mode='grayscale',
    class_mode='binary'
)

Found 7178 images belonging to 7 classes.


In [6]:
from keras import regularizers

model =  tf.keras.Sequential([
    tf.keras.layers.Conv2D(32,(3,3), activation='relu' ,input_shape = (48,48,1)),
    tf.keras.layers.Conv2D(64,(3,3),  activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Dropout(0.25),

    tf.keras.layers.Conv2D(128,(3,3), activation='relu' , kernel_regularizer=regularizers.l2(0.01)),
    tf.keras.layers.Conv2D(256,(3,3), activation='relu', kernel_regularizer=regularizers.l2(0.01)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Dropout(0.25),

    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(1024,activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(7, activation='softmax')
]) 

In [7]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 46, 46, 32)        320       
                                                                 
 conv2d_1 (Conv2D)           (None, 44, 44, 64)        18496     
                                                                 
 batch_normalization (BatchN  (None, 44, 44, 64)       256       
 ormalization)                                                   
                                                                 
 max_pooling2d (MaxPooling2D  (None, 22, 22, 64)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 22, 22, 64)        0         
                                                                 
 conv2d_2 (Conv2D)           (None, 20, 20, 128)       7

In [8]:
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [9]:
early=tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True
)

In [10]:
reduce_lr=tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.2,
    patience=6,
    verbose=1,
    min_delta=0.0001
    )

In [11]:
history=model.fit(
    train_generator,
    validation_data=test_generator,
    callbacks=[early,reduce_lr],
    epochs=100
)

c:\Users\Sivaprasath\anaconda3\envs\akaike\lib\site-packages\keras\preprocessing\image.py:1863: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn(
c:\Users\Sivaprasath\anaconda3\envs\akaike\lib\site-packages\keras\preprocessing\image.py:1873: UserWarning: This ImageDataGenerator specifies `featurewise_std_normalization`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn(


Epoch 1/100
449/449 [==============================] - 590s 1s/step - loss: 3.6565 - accuracy: 0.2696 - val_loss: 3.6821 - val_accuracy: 0.2154 - lr: 0.0010
Epoch 2/100
449/449 [==============================] - 199s 444ms/step - loss: 1.8768 - accuracy: 0.3388 - val_loss: 2.0489 - val_accuracy: 0.3396 - lr: 0.0010
Epoch 3/100
449/449 [==============================] - 206s 458ms/step - loss: 1.6844 - accuracy: 0.3618 - val_loss: 2.1695 - val_accuracy: 0.3197 - lr: 0.0010
Epoch 4/100
449/449 [==============================] - 205s 457ms/step - loss: 1.6129 - accuracy: 0.3865 - val_loss: 1.5370 - val_accuracy: 0.4025 - lr: 0.0010
Epoch 5/100
449/449 [==============================] - 214s 477ms/step - loss: 1.5822 - accuracy: 0.4033 - val_loss: 1.6048 - val_accuracy: 0.3945 - lr: 0.0010
Epoch 6/100
449/449 [==============================] - 207s 460ms/step - loss: 1.5458 - accuracy: 0.4177 - val_loss: 1.9742 - val_accuracy: 0.3070 - lr: 0.0010
Epoch 7/100
449/449 [======================

In [12]:
model.save('emodet.h5')